In [ ]:
# 1. 모델 정의
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# 모델 정의
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        layers.Embedding(vocab_size, embedding_dim, batch_input_shape=(batch_size, None)),
        layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        layers.Dense(vocab_size, activation='softmax')
    ])
    return model

In [ ]:
# 2. 데이터셋 로드 및 전처리
from tensorflow.keras import utils
# Shakespeare의 텍스트 데이터를 텐서플로우 데이터셋에서 불러온다.
path_to_file = utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# 데이터셋 읽기
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 텍스트의 길이 확인
print('Length of text : {} characters'.format(len(text))) # 1115394

# 고유 문자 확인
vocab = sorted(set(text)) # 65
print('{} unique characters'.format(len(vocab)))

Length of text : 1115394 characters
65 unique characters


In [ ]:
# 3. 모델 생성
# Vocabulary 길이
vocab_size = len(vocab) # 65

# 임베딩 차원
embedding_dim = 256

model = build_model(vocab_size=vocab_size,
                    embedding_dim = embedding_dim,
                    rnn_units=1024,
                    batch_size=64)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 4. 문자 인덱싱 및 텐서 변환
import numpy as np

# 문자 -> 인덱스
idx2char = np.array(vocab)

# 인덱스 -> 문자
char2idx = {u:i for i, u in enumerate(vocab)}

# 텍스트 -> 숫자
text_as_int = np.array([char2idx[c] for c in text])

print(text_as_int) # 텍스트를 숫자로 바꾼 데이터 출력해 봄
print(text[:3], text[-3:])

[18 47 56 ... 45  8  0]
Fir g.



In [ ]:
# 5. 시퀀스 및 배치 생성
import tensorflow as tf

# 시퀀스 길이 정의
seq_length = 100
examples_per_epoch = len(text) // seq_length # 11153

# 데이터셋 생성
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(examples_per_epoch).batch(64, drop_remainder=True)

In [ ]:
# 6. 훈련 정의
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
# 7. 체크포인트 설정
checkpoint_callback= tf.keras.callbacks.ModelCheckpoint(filepath='./checkpoints/ckpt_{epoch}', save_best_only=True)

In [ ]:
# 8. 모델 학습
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
172/172 [==============================] - ETA: 0s - loss: 2.5777

172/172 [==============================] - 1441s 8s/step - loss: 2.5777
Epoch 2/100
172/172 [==============================] - ETA: 0s - loss: 1.8842

172/172 [==============================] - 1366s 8s/step - loss: 1.8842
Epoch 3/100
172/172 [==============================] - ETA: 0s - loss: 1.6306

172/172 [==============================] - 1364s 8s/step - loss: 1.6306
Epoch 4/100
172/172 [==============================] - ETA: 0s - loss: 1.4975

172/172 [==============================] - 1354s 8s/step - loss: 1.4975
Epoch 5/100
172/172 [==============================] - ETA: 0s - loss: 1.4175

172/172 [==============================] - 1348s 8s/step - loss: 1.4175
Epoch 6/100
172/172 [==============================] - ETA: 0s - loss: 1.3604

172/172 [==============================] - 1352s 8s/step - loss: 1.3604
Epoch 7/100
172/172 [==============================] - ETA: 0s - loss: 1.3153

172/172 [==============================] - 1354s 8s/step - loss: 1.3153
Epoch 8/100
 45/172 [======>.......................] - ETA: 16:37 - loss: 1.2768

In [ ]:
# 9. 체크포인트 복원 및 테스트 함수
def generate_text(model, start_string, num_generate=300, temperature=1.0):
    model_for_inference= build_model(vocab_size, embedding_dim, 1024, batch_size=1) # 추론 모델 정의
    model_for_inference.set_weights(model.get_weights()) # 현재 학습된 모델의 파라미터를 추론 모델에 set
    model_for_inference.reset_states()
    input_data = [char2idx[c] for c in start_string] # 입력 문장을 숫자 리스트로
    input_data = tf.expand_dims(input_data, 0)

    text_generated = []
    for i in range(num_generate):
        predictions = model_for_inference(input_data) # 예측 생성
        predictions = tf.squeeze(predictions, 0) # 차원을 축소해서 불필요한 차원을 제거
        # 예측의 분포를 조절, 온도가 낮으면 더 결정적인 예측, 온도가 높으면 무작위적인 예측
        predictions = predictions / temperature
        # 샘플링된 다음 문자의 인덱스
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_data = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return ''.join(text_generated)

In [ ]:
# 10. 체크포인트 복원 및 테스트
input_text = input('로미오와 줄리엣 소설의 문장을 입력하세요.')
output = generate_text(model, input_text, temperature=1.0)
print(output)